In [4]:
# Project Plan:
# 1. Setup or get video input
# 2. Build recognition elements

In [7]:
#Install Dependencies
import cv2 
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import dlib
import time
from scipy.spatial import distance

In [ ]:

class PoseAnalyzer:

    def __init__(self,head_range=(115,130),hip_angle_threshold=45):

        self.head_range = head_range
        self.hip_angle_threshold = hip_angle_threshold

    def calculate_angle(self,a,b,c):
        ab = np.array([a.x-b.x,a.y-b.y])
        bc = np.array([b.x-c.x,b.y-c.y])

        dot_product = np.dot(ab,bc)
        magnitude_ab = np.linalg.norm(ab)
        magnitude_bc = np.linalg.norm(bc)

        cos_angle = dot_product / (magnitude_ab * magnitude_bc)
    
        angle = np.arccos(np.clip(cos_angle,-1.0,1.0))
        angle_deg = np.degrees(angle)

        return angle_deg
    
    def is_head_tilted(self,shoulder_left,shoulder_right,nose):

        #Checks if head is tilted, based on the range we consider not tilted
        tilt_angle = self.calculate_angle(shoulder_left,shoulder_right,nose)
        is_tilted = not (self.head_range[0] <= tilt_angle <= self.head_range[1])
        return tilt_angle, is_tilted


    def is_standing(self,knee_left,hip_left,shoulder_left,knee_right,hip_right,shoulder_right):
        left_hip_angle = self.calculate_angle(knee_left,hip_left,shoulder_left)
        right_hip_angle = self.calculate_angle(knee_right,hip_right,shoulder_right)

        is_standing = left_hip_angle < self.hip_angle_threshold and right_hip_angle < self.hip_angle_threshold
        return left_hip_angle,right_hip_angle,is_standing
        



class EyeStateDetector:

    def __init__(self,ear_threshold):
        self.ear_threshold = ear_threshold
        self.left_eye_closed_start = None
        self.right_eye_closed_start = None
        self.left_eye_closed_duration = 0
        self.right_eye_closed_duration = 0
        self.eye_closed_duration = 0

    def eye_aspect_ratio(self,eye):
        #Vertical landmarks
        A = distance.euclidean(eye[1],eye[5])
        B = distance.euclidean(eye[2],eye[4])
        #Horizontal landmarks
        C = distance.euclidean(eye[0],eye[3])
        ea_ratio = (A+B)/(2*C)
        return ea_ratio

    def update_eye_status(self,left_eye,right_eye,current_time):
       
        #Calculate EAR for both eyes
        left_ear = self.eye_aspect_ratio(left_eye)
        right_ear = self.eye_aspect_ratio(right_eye)

        #Set start time of closed eye
        if left_ear < self.ear_threshold:
            if self.left_eye_closed_start is None:
                self.left_eye_closed_start = current_time
        else:
            self.left_eye_closed_start = None

        if right_ear < self.ear_threshold:
            if self.right_eye_closed_start is None:
                self.right_eye_closed_start = current_time
        else:
            self.right_eye_closed_start = None

        #Calculate duration of closed eyes
        if self.left_eye_closed_start:
            self.left_eye_closed_duration = current_time - self.left_eye_closed_start
        else:
            self.left_eye_closed_duration = 0

        if self.right_eye_closed_start:
            self.right_eye_closed_duration = current_time - self.right_eye_closed_start
        else:
            self.right_eye_closed_duration = 0
        
        #Check for sleepiness
        if (self.left_eye_closed_duration > 2 or self.right_eye_closed_duration > 2):
            self.eye_closed_duration += 1 
        else:
            self.eye_closed_duration = 0

        return left_ear, right_ear, self.eye_closed_duration


In [12]:
#Setting up MediaPipe Pose model for body pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

#Initializing drawing utils
mp_drawing = mp.solutions.drawing_utils

In [ ]:
sleepines_detector = EyeStateDetector(ear_threshold=0.2)
pose_analyzer = PoseAnalyzer(
    #Head between 115 and 130 is straight, everything else considered tilt,
    head_range=(115,130),hip_angle_threshold=45)

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
    
    current_time = time.time()
    frame = cv2.flip(frame,1)
    rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    results_pose = pose.process(rgb_frame)
    faces = detector(rgb_frame)

    if results_pose.pose_landmarks:

        mp_drawing.draw_landmarks(frame,results_pose.pose_landmarks,mp_pose.POSE_CONNECTIONS)
        landmarks = results_pose.pose_landmarks.landmark

        points = {
            "Left Shoulder": landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER],
            "Right Shoulder": landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
            "Left Hip": landmarks[mp_pose.PoseLandmark.LEFT_HIP],
            "Right Hip": landmarks[mp_pose.PoseLandmark.RIGHT_HIP],
            "Left Knee": landmarks[mp_pose.PoseLandmark.LEFT_KNEE],
            "Right Knee": landmarks[mp_pose.PoseLandmark.RIGHT_KNEE],
            "Nose": landmarks[mp_pose.PoseLandmark.NOSE],

        }


        #Understand if person is standing or not
        left_hip_angle,right_hip_angle,is_standing = pose_analyzer.is_standing(
            points["Left Knee"],
            points["Left Hip"],
            points["Left Shoulder"],
            points["Right Knee"],
            points["Right Hip"],
            points["Right Shoulder"]
        )

        #Understand if the head is tilted

        tilt_angle, is_tilted = pose_analyzer.is_head_tilted(
            points["Left Shoulder"],
            points["Right Shoulder"],
            points["Nose"]
        )

        #Print our metrics and conclusions in the top left border of the screen
        cv2.putText(frame, f'Left Hip Angle: {int(left_hip_angle)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Right Hip Angle: {int(right_hip_angle)}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Is Standing: {is_standing}', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Head Tilt Angle: {int(tilt_angle)}', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Head Tilted: {is_tilted}', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

  
        ##Print identified landmarks. Not needed, more to just see what the model is "seeing"
        for point_name, point in points.items():
            x,y = int(point.x * frame.shape[1]), int(point.y * frame.shape[0])
            cv2.circle(frame,(x,y),5,(0,255,0),-1)
            cv2.putText(frame,point_name, (x,y),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)

    if faces:
        for face in faces:
            landmarks = predictor(rgb_frame,face)

            right_eye = [
                (landmarks.part(i).x,landmarks.part(i).y)
                for i in range(36,42)
            ]

            left_eye = [
                (landmarks.part(i).x,landmarks.part(i).y)
                for i in range(42,48)
            ]

            left_ear,right_ear = sleepines_detector.update_eye_status(left_eye,right_eye,current_time)


            cv2.putText(frame, f"Left EAR: {left_ear:.2f}", (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Right EAR: {right_ear:.2f}", (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            x1,y1,x2,y2 = (face.left(),face.top(),face.right(),face.bottom())
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)

    cv2.imshow("Pose & Face Landmarks",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

cap.release()
cv2.destroyAllWindows()

In [ ]:
#### For a proper sleepiness detector we need to consider the factor time more. 
### Example: a person yawned once in the last minute and has a certain head tilted angle. 
### If a person is standing and has the eyes closed, it could for example be that they are standing in the sun and enjoying it. 